#the customers who individually ordered on 3 consecutive days where each day’s total order was more than 50 in value.

In [1]:
import pandas as pd

In [2]:
from datetime import timedelta

In [3]:
# read the data into a pandas dataframe
df = pd.read_csv(r"C:\Users\Yamin Hossain\Downloads\superstore_final_cleaned_data.csv")

In [4]:
# convert Order_Date to datetime format
df['Order_Date'] = pd.to_datetime(df['Order_Date'])

In [5]:
df

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales
0,1,CA-2017-152156,2017-08-11,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.960
1,2,CA-2017-152156,2017-08-11,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.940
2,14,CA-2017-161389,2017-05-12,2017-10-12,Standard Class,IM-15070,Irene Maddox,Consumer,United States,Seattle,Washington,98103.0,West,OFF-BI-10003656,Office Supplies,Binders,Fellowes PB200 Plastic Comb Binding Machine,407.976
3,36,CA-2017-117590,2017-08-12,2017-10-12,First Class,GH-14485,Gene Hale,Corporate,United States,Richardson,Texas,75080.0,Central,TEC-PH-10004977,Technology,Phones,GE 30524EE4,1097.544
4,37,CA-2017-117590,2017-08-12,2017-10-12,First Class,GH-14485,Gene Hale,Corporate,United States,Richardson,Texas,75080.0,Central,FUR-FU-10003664,Furniture,Furnishings,"Electrix Architects Clamp-On Swing Arm Lamp, B...",190.920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2671,9750,CA-2017-158358,2017-04-03,2017-08-03,Standard Class,EM-13810,Eleni McCrary,Corporate,United States,Dover,New Hampshire,3820.0,East,TEC-AC-10002567,Technology,Accessories,Logitech G602 Wireless Gaming Mouse,159.980
2672,9753,US-2017-158680,2017-01-07,2017-02-07,First Class,NZ-18565,Nick Zandusky,Home Office,United States,Seattle,Washington,98115.0,West,OFF-BI-10000145,Office Supplies,Binders,Zipper Ring Binder Pockets,2.496
2673,9761,CA-2018-159135,2018-04-11,2018-08-11,Standard Class,KM-16375,Katherine Murray,Home Office,United States,Springdale,Arkansas,72762.0,South,OFF-AR-10000657,Office Supplies,Art,"Binney & Smith inkTank Desk Highlighter, Chise...",4.300
2674,9773,CA-2017-104983,2017-03-07,2017-03-07,Same Day,FM-14215,Filia McAdams,Corporate,United States,Philadelphia,Pennsylvania,19143.0,East,OFF-PA-10000167,Office Supplies,Paper,Xerox 1925,123.920


In [6]:
# group the data by Customer_Name and Order_Date and calculate the daily total sales
df_grouped = df.groupby(['Customer_Name', 'Order_Date'])['Sales'].sum().reset_index()

In [10]:
df_grouped

,Customer_Name,Order_Date,Sales
0,Aaron Bergman,2015-07-03,309.592
1,Aaron Bergman,2017-10-11,563.940
2,Aaron Smayling,2018-01-08,1476.270
3,Aaron Smayling,2018-02-01,740.214
4,Aaron Smayling,2018-03-10,171.288
...,...,...,...
1319,Zuschuss Carroll,2017-08-04,1616.704
1320,Zuschuss Carroll,2018-06-11,29.690
1321,Zuschuss Donatelli,2017-03-04,331.080
1322,Zuschuss Donatelli,2017-05-05,16.720


In [8]:
# sort the data by Customer_Name and Order_Date
df_grouped = df_grouped.sort_values(['Customer_Name', 'Order_Date'])

In [11]:
# create a new column to store the previous day's date for each order
df_grouped['Prev_Date'] = df_grouped.groupby('Customer_Name')['Order_Date'].shift(1)

In [12]:
# create a new column to store the difference in days between the current and previous orders
df_grouped['Day_Diff'] = (df_grouped['Order_Date'] - df_grouped['Prev_Date']).dt.days

In [13]:
# create a new column to store the cumulative sum of sales for each customer
df_grouped['Cum_Sales'] = df_grouped.groupby('Customer_Name')['Sales'].cumsum()

In [14]:
# filter the data to include only orders with a sales value greater than 50
df_filtered = df_grouped[df_grouped['Sales'] > 50]

In [15]:
# group the data by Customer_Name and Day_Diff and count the number of consecutive days with sales greater than 50
df_consecutive = df_filtered.groupby(['Customer_Name', 'Day_Diff']).size().reset_index(name='Consecutive_Days')

In [16]:
# filter the data to include only customers with 3 or more consecutive days with sales greater than 50
df_result = df_consecutive[df_consecutive['Consecutive_Days'] >= 3]

In [17]:
df_result

,Customer_Name,Day_Diff,Consecutive_Days


#the maximum number of days for which total sales on each day kept rising.

In [18]:
# Group the data by date and calculate the total sales for each date
daily_sales = df.groupby("Order_Date")["Sales"].sum()

In [19]:
# Calculate the difference in sales between consecutive days
daily_diff = daily_sales.diff()

In [20]:
# Find the maximum number of days for which sales kept rising
max_rising_days = 0
current_rising_days = 0
for diff in daily_diff:
    if diff > 0:
        current_rising_days += 1
        max_rising_days = max(max_rising_days, current_rising_days)
    else:
        current_rising_days = 0

print("Maximum number of days for which total sales kept rising:", max_rising_days)

Maximum number of days for which total sales kept rising: 3
